In [13]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine('sqlite:///myData.db')

dfo  = pd.read_csv('orders.csv')
dft= pd.read_csv('tarikhche.csv')
dfp  = pd.read_excel('product.xlsx')

dfo.to_sql('orders', con=engine, if_exists='replace', index= False)
dft.to_sql('product', con=engine, if_exists='replace', index= False)
dfp.to_sql('tarikhche', con=engine, if_exists='replace', index= False)

import sqlite3
conn = sqlite3.connect('myData.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM orders")
print(cursor.fetchmany(10))
conn.close()


OperationalError: (sqlite3.OperationalError) database is locked
[SQL: 
DROP TABLE orders]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [17]:
orders = pd.read_sql_query("SELECT * FROM orders", conn)
products = pd.read_sql_query("SELECT * FROM product", conn)
tarikhche = pd.read_sql_query("SELECT * FROM tarikhche", conn)

print(orders.columns)
print(products.columns)
print(tarikhche.columns)

merged = pd.merge(orders, products, left_on="ID_Item", right_on="product_id", how="inner")
final = pd.merge(merged, tarikhche, left_on="product_id", right_on="id", how="inner")

final.to_sql('final_table', conn, if_exists='replace', index= False)
print(final.head())


Index(['ID_Order', 'ID_Customer', 'ID_Item', 'DateTime_CartFinalize',
       'Amount_Gross_Order', 'city_name_fa', 'Quantity_item'],
      dtype='object')
Index(['id', 'product_variant_id', 'selling_price', 'rrp_price', 'base_price',
       'buy_price', 'order_limit', 'start_at', 'end_at', 'tags',
       'show_in_price_history', 'active', 'created_at', 'product_id',
       'marketplace_seller_id'],
      dtype='object')
Index(['id', 'product_title_fa', 'product_title_en', 'url_code', 'title_alt',
       'category_title_fa', 'category_keywords', 'brand_name_fa',
       'brand_name_en', 'product_attributes'],
      dtype='object')
   ID_Order  ID_Customer  ID_Item    DateTime_CartFinalize  \
0  21731202      5844553   771199  2018-08-18 05:12:23.000   
1  21731202      5844553   771199  2018-08-18 05:12:23.000   
2  21731202      5844553   771199  2018-08-18 05:12:23.000   
3  21731202      5844553   771199  2018-08-18 05:12:23.000   
4  21731202      5844553   771199  2018-08-18 05:12:2

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
df = pd.read_sql_query("SELECT * FROM 'final_table'", conn)
print(df.head())

   ID_Order  ID_Customer  ID_Item    DateTime_CartFinalize  \
0  21731202      5844553   771199  2018-08-18 05:12:23.000   
1  21731202      5844553   771199  2018-08-18 05:12:23.000   
2  21731202      5844553   771199  2018-08-18 05:12:23.000   
3  21731202      5844553   771199  2018-08-18 05:12:23.000   
4  21731202      5844553   771199  2018-08-18 05:12:23.000   

   Amount_Gross_Order city_name_fa  Quantity_item      id_x  \
0             35000.0         مشهد            1.0  18090763   
1             35000.0         مشهد            1.0  18555182   
2             35000.0         مشهد            1.0  18619036   
3             35000.0         مشهد            1.0  18619056   
4             35000.0         مشهد            1.0  11459454   

   product_variant_id  selling_price  ...    id_y  \
0             1921828          80000  ...  771199   
1             1921828         100000  ...  771199   
2             1921828          79000  ...  771199   
3             1921828          69000

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8631 entries, 0 to 8630
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID_Order               8631 non-null   int64  
 1   ID_Customer            8631 non-null   int64  
 2   ID_Item                8631 non-null   int64  
 3   DateTime_CartFinalize  8631 non-null   object 
 4   Amount_Gross_Order     8631 non-null   float64
 5   city_name_fa           8631 non-null   object 
 6   Quantity_item          8631 non-null   float64
 7   id_x                   8631 non-null   int64  
 8   product_variant_id     8631 non-null   int64  
 9   selling_price          8631 non-null   int64  
 10  rrp_price              8631 non-null   int64  
 11  base_price             8631 non-null   object 
 12  buy_price              8631 non-null   object 
 13  order_limit            8631 non-null   int64  
 14  start_at               8631 non-null   object 
 15  end_

In [34]:
df.describe()

,ID_Order,ID_Customer,ID_Item,Amount_Gross_Order,Quantity_item,id_x,product_variant_id,selling_price,rrp_price,order_limit,show_in_price_history,active,product_id,marketplace_seller_id,id_y
count,8.631000e+03,8.631000e+03,8631.000000,8.631000e+03,8631.000000,8.631000e+03,8.631000e+03,8.631000e+03,8.631000e+03,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000,8631.000000
mean,1.467166e+07,4.413541e+06,765270.465647,8.708823e+05,1.446646,1.443115e+07,1.530280e+06,5.350331e+06,5.516704e+06,3.735141,0.999884,0.077395,765270.465647,19603.292782,765270.465647
std,4.302711e+06,2.296436e+06,22212.266960,2.104158e+06,1.347724,2.170199e+06,1.284680e+05,1.444719e+07,1.446510e+07,9.319702,0.010764,0.267233,22212.266960,26078.646298,22212.266960
min,1.111461e+07,4.708560e+05,724078.000000,8.257000e+03,1.000000,9.864232e+06,1.338024e+06,1.000000e+02,1.000000e+02,1.000000,0.000000,0.000000,724078.000000,1.000000,724078.000000
25%,1.196845e+07,2.783254e+06,753430.000000,3.500000e+04,1.000000,1.272531e+07,1.428851e+06,3.390000e+05,3.500000e+05,1.000000,1.000000,0.000000,753430.000000,1555.000000,753430.000000
50%,1.276387e+07,4.922098e+06,762692.000000,3.200000e+05,1.000000,1.416860e+07,1.496274e+06,1.244000e+06,1.500000e+06,2.000000,1.000000,0.000000,762692.000000,6891.000000,762692.000000
75%,1.379976e+07,6.701614e+06,771199.000000,6.280000e+05,1.000000,1.601406e+07,1.611563e+06,2.900000e+06,3.000000e+06,3.000000,1.000000,0.000000,771199.000000,16135.000000,771199.000000
max,2.484075e+07,7.265505e+06,827122.000000,2.094550e+07,10.000000,1.875155e+07,1.976535e+06,7.160000e+08,7.160000e+08,120.000000,1.000000,1.000000,827122.000000,88092.000000,827122.000000


In [48]:
cleaned_df = df[["product_title_fa", "city_name_fa", "category_keywords", "brand_name_en"]]
cleaned_df

,product_title_fa,city_name_fa,category_keywords,brand_name_en
0,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous
1,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous
2,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous
3,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous
4,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous
...,...,...,...,...
8626,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous
8627,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous
8628,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous
8629,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous


In [49]:
cleaned_df["Combined_Features"] = cleaned_df["product_title_fa"] + " " + cleaned_df["city_name_fa"] + " " + cleaned_df["category_keywords"] + " " + cleaned_df["brand_name_en"]

cleaned_df["Combined_Features"] = cleaned_df["Combined_Features"].str.replace(",", "")

cleaned_df

C:\Users\Student\AppData\Local\Temp\ipykernel_3888\4071954745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["Combined_Features"] = cleaned_df["product_title_fa"] + " " + cleaned_df["city_name_fa"] + " " + cleaned_df["category_keywords"] + " " + cleaned_df["brand_name_en"]
C:\Users\Student\AppData\Local\Temp\ipykernel_3888\4071954745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df["Combined_Features"] = cleaned_df["Combined_Features"].str.replace(",", "")


,product_title_fa,city_name_fa,category_keywords,brand_name_en,Combined_Features
0,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous,بادکنک لاتکس مدل I Love You بسته 10 عددی مشهد...
1,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous,بادکنک لاتکس مدل I Love You بسته 10 عددی مشهد...
2,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous,بادکنک لاتکس مدل I Love You بسته 10 عددی مشهد...
3,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous,بادکنک لاتکس مدل I Love You بسته 10 عددی مشهد...
4,بادکنک لاتکس مدل I Love You بسته 10 عددی,مشهد,ابزار مهمانی Party-Tools,Miscellaneous,بادکنک لاتکس مدل I Love You بسته 10 عددی مشهد...
...,...,...,...,...,...
8626,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW ...
8627,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW ...
8628,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW ...
8629,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW,اهواز,سرویس غذاخوری Dinnerware-Sets,Miscellaneous,سرویس غذاخوری 16 پارچه استون ور مدل ST7213-BW ...


In [50]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
feature_vector = vectorizer.fit_transform(cleaned_df["Combined_Features"])

feature_vector

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 109103 stored elements and shape (8631, 723)>

In [51]:
similarities = cosine_similarity(feature_vector)

similarities

array([[1.        , 1.        , 1.        , ..., 0.12262787, 0.12262787,
        0.12262787],
       [1.        , 1.        , 1.        , ..., 0.12262787, 0.12262787,
        0.12262787],
       [1.        , 1.        , 1.        , ..., 0.12262787, 0.12262787,
        0.12262787],
       ...,
       [0.12262787, 0.12262787, 0.12262787, ..., 1.        , 1.        ,
        1.        ],
       [0.12262787, 0.12262787, 0.12262787, ..., 1.        , 1.        ,
        1.        ],
       [0.12262787, 0.12262787, 0.12262787, ..., 1.        , 1.        ,
        1.        ]])

In [62]:
product = input("Enter product title: ")

product_idx = cleaned_df[cleaned_df["product_title_fa"].str.contains(product, case= False, na = False)].index[0]

recomm = np.c_[cleaned_df["product_title_fa"], similarities[product_idx]]


num_recom = 3
sorted (recomm, key=lambda m:m[1], reverse=True)[1:num_recom +1]

[array(['ماشین بازی کنترلی ام اس اچ مدل Crazy Car', 1.0], dtype=object),
 array(['ماشین بازی کنترلی ام اس اچ مدل Crazy Car', 1.0], dtype=object),
 array(['ماشین بازی کنترلی ام اس اچ مدل Crazy Car', 1.0], dtype=object)]